# Prototype Aus-wide saltmarsh

In [1]:
# load imports/set up
%run 'saltmarsh_prototype_imports.ipynb'

ncpus = 15


In [ ]:
## Training data/covariate data sampling

## Load training data
- 'Class' field includes all classes: Land [0], Water [1], Saltmarsh [5], Seagrass [6,9], Tidal flat [11]
- 'wetland' field is intertidal wetland [1] or not [0]

In [2]:
# load shapefile
training_data = gpd.read_file("training_data/prelim_saltmarsh_training.shp")

'''
# subsamle for testing
training_data['random'] = np.random.uniform(0, 1, training_data.shape[0])
training_data = training_data[training_data.random > 0.997]
training_data = training_data.reset_index(drop=True)
'''

print(training_data.head())
print(training_data.shape[0])
print(training_data.crs)

# Plot training data in an interactive map
#map_shapefile(training_data, attribute="wetland")

   DateEnd  Class  DateSta  wetland  site                     geometry
0      NaN      0      NaN        0     1  POINT (153.19066 -25.25695)
1      NaN      0      NaN        0     2  POINT (153.13410 -25.39304)
2      NaN      0      NaN        0     3  POINT (153.26061 -25.07794)
3      NaN      0      NaN        0     4  POINT (151.65435 -24.00486)
4      NaN      0      NaN        0     5  POINT (151.67591 -24.01595)
877
epsg:4326


## Extract training data/covariates
- we can build these further based on existing work + datacube dependencies
- also this will transition to extracting data specific to the time periods of insitu data validity

In [3]:
# Set up our inputs to collect_training_data
time = ("2019-01-01", "2020-12-31")
zonal_stats = None
return_coords = False # for now

field = "site"

# Set up the data cube inputs
measurements = ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']
resolution = (-30, 30)
output_crs = 'epsg:3577'

# Generate a new datacube query object
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs,
}

### features/indices to add

In [4]:
def feature_layers(query):
    #connect to the datacube
    dc = datacube.Datacube(app='aust_saltmarsh')
    
    #load ls8 data
    ds = dc.load(product='ga_ls8c_ard_3',
                 **query)
    
    # add reflectance medians
    ref_med = ds.median('time')
    
    # Add reflectance variance metrics
    ref_sdev = ds.std('time')
    ref_sdev_names = dict(zip(list(ref_sdev.keys()), [x + '_std' for x in list(ref_sdev.keys())]))
    ref_sdev = ref_sdev.rename_vars(name_dict=ref_sdev_names)
    
    # Calculate relevant band indices
    indices = calculate_indices(ref_med,
                                index=['NDVI', 'NDWI'], # add others later
                                drop=True,
                                collection='ga_ls_3')
    '''
    more things to add:
    - merged land DEM/intertidal DEM/ocean bathymetry
    - slope
    - temperature
    '''
      
    # Merge results into single dataset 
    result = xr.merge([ref_med, ref_sdev, indices], compat='override')
    
    return result

### Sample the covariates

In [5]:
column_names, covariate_samples = collect_training_data(
    gdf=training_data,
    dc_query=query,
    ncpus=ncpus,
    return_coords=return_coords,
    field=field,
    zonal_stats=zonal_stats,
    feature_func=feature_layers)

print(column_names)
print('')
print(covariate_samples[:,])

  0%|          | 0/877 [00:00<?, ?it/s]

Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']


Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']
Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']
Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']
Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']
Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_swir_1']
Dropping bands ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir', 'nbart_s

In [6]:
# append any other class fields needed

# merge with additional data (convert to pd for easy merge)
covariate_samples_df = pd.DataFrame(covariate_samples, columns=column_names)
#print(covariate_samples_df)
model_input_merge = covariate_samples_df.merge(training_data, how='inner', on='site').drop(['geometry','id'], axis=1)
#print(model_input_merge)

# create vars for modelling stage
model_input = model_input_merge.to_numpy()
column_names = model_input_merge.columns.values

print(column_names)
print('')
print(model_input)

['site' 'nbart_coastal_aerosol' 'nbart_blue' 'nbart_green' 'nbart_red'
 'nbart_nir' 'nbart_swir_1' 'nbart_coastal_aerosol_std' 'nbart_blue_std'
 'nbart_green_std' 'nbart_red_std' 'nbart_nir_std' 'nbart_swir_1_std'
 'NDVI' 'NDWI' 'DateEnd' 'Class' 'DateSta' 'wetland']

[[8.0000000e+00 5.7200000e+02 5.7800000e+02 ... 0.0000000e+00
            nan 0.0000000e+00]
 [7.0000000e+00 7.3400000e+02 8.0100000e+02 ... 0.0000000e+00
            nan 0.0000000e+00]
 [9.0000000e+00 5.2600000e+02 5.5200000e+02 ... 0.0000000e+00
            nan 0.0000000e+00]
 ...
 [8.7600000e+02 6.8200000e+02 7.2300000e+02 ... 1.1000000e+01
  1.6233948e+12 1.0000000e+00]
 [8.7300000e+02 6.7300000e+02 7.2100000e+02 ... 1.1000000e+01
  1.6280604e+12 1.0000000e+00]
 [8.7700000e+02 1.1515000e+03 1.2380000e+03 ... 1.1000000e+01
  1.5970428e+12 1.0000000e+00]]


In [7]:
# Set the name and location of the output file
output_file = "training_data/prelim_covariate_data.txt"
# Export files to disk
np.savetxt(output_file, model_input, header=" ".join(column_names), fmt="%4f")